# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint

pprint.pprint(
    [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
)

[{'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7001,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9041,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x0000024376F3ABA0>,
  'dse_version': None,
  'dse_workload': None,
  'dse_workloads': None,
  'endpoint': <DefaultEndPoint: cassandra-node-1.mavasbel.vpn.itam.mx:9041>,
  'host_id': UUID('e113825e-cadb-43a2-bdec-793fd0cd67ee'),
  'is_up': True,
  'listen_address': '172.19.0.2',
  'listen_port': 7001,
  'lock': <unlocked _thread.RLock object owner=0 count=0 at 0x0000024374D85340>,
  'release_version': '5.0.6'},
 {'_datacenter': 'dc1',
  '_rack': 'rack1',
  'broadcast_address': '10.15.20.2',
  'broadcast_port': 7002,
  'broadcast_rpc_address': '10.15.20.2',
  'broadcast_rpc_port': 9042,
  'conviction_policy': <cassandra.policies.SimpleConvictionPolicy object at 0x0000024376F2D810>,
  'dse_version': None,
  'dse_workload': None,
  'dse_wo

In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [20]:
from faker import Faker

fake = Faker()

In [21]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [10]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [11]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'North Daniel', 'user_id': UUID('63065224-f174-4cdc-be5b-a64f685684f2'), 'last_access': datetime.datetime(2026, 1, 11, 19, 9, 36, 708000), 'session_duration': 3593, 'username': 'cherylperez'}
{'city': 'Port Rebecca', 'user_id': UUID('417930eb-ae2c-4a54-8e36-f6ee8e08a69f'), 'last_access': datetime.datetime(2026, 1, 11, 19, 9, 36, 725000), 'session_duration': 3170, 'username': 'jamesgibbs'}
{'city': 'Lukefort', 'user_id': UUID('50a34ebb-b32e-45e1-90eb-3dbc1822879c'), 'last_access': datetime.datetime(2026, 1, 11, 19, 9, 36, 710000), 'session_duration': 451, 'username': 'karenfuentes'}
{'city': 'East Anna', 'user_id': UUID('895d6404-8993-4a7e-a755-25882ff88059'), 'last_access': datetime.datetime(2026, 1, 11, 19, 9, 36, 732000), 'session_duration': 1282, 'username': 'gclements'}
{'city': 'Fischermouth', 'user_id': UUID('de5b4289-0dd5-4dff-ac1a-44500b22e485'), 'last_access': datetime.datetime(2026, 1, 11, 19, 9, 36, 712000), 'session_duration': 2829, 'username': 'gregory27'}
{'city'

In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'North Daniel'}
{'city': 'Port Rebecca'}
{'city': 'Lukefort'}
{'city': 'East Anna'}
{'city': 'Fischermouth'}
{'city': 'North Elizabeth'}
{'city': 'North Xaviermouth'}
{'city': 'Rebeccabury'}
{'city': 'Serranoport'}
{'city': 'Johnsonhaven'}
{'city': 'West Brenda'}
{'city': 'Bendershire'}
{'city': 'Catherineburgh'}
{'city': 'East Rebeccaville'}
{'city': 'Williamsstad'}
{'city': 'East Tonyton'}
{'city': 'Tracyport'}
{'city': 'Michaelaport'}
{'city': 'West Jeffreyville'}
{'city': 'New Michelleton'}
{'city': 'Port Casey'}
{'city': 'North Tina'}
{'city': 'North Amandaburgh'}
{'city': 'Lake Jennifer'}
{'city': 'Williamsborough'}
{'city': 'South Jesse'}
{'city': 'New Jacqueline'}
{'city': 'Port Paula'}
{'city': 'Nicholasbury'}
{'city': 'Port Samuel'}
{'city': 'Burkechester'}
{'city': 'Allenburgh'}
{'city': 'South Alyssastad'}
{'city': 'New Kaylaview'}
{'city': 'East Clifford'}
{'city': 'New Jennifertown'}
{'city': 'Rodriguezberg'}
{'city': 'Jonesstad'}
{'city': 'Lake Andrew'}
{'city':

### Find nodes storing random data

In [22]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Smithstad',
 'last_access': datetime.datetime(2026, 1, 11, 19, 13, 3, 470000),
 'session_duration': 1172,
 'user_id': UUID('d7d26094-383b-4ed9-8044-907f72f1e610'),
 'username': 'whiterichard'}
Nodes storing 'Smithstad':
 - Host: 10.15.20.2, Gossip Port: 7002
 - Host: cassandra-node-1.mavasbel.vpn.itam.mx, Gossip Port: 7001
 - Host: 10.15.20.2, Gossip Port: 7003


### ORM-like

In [14]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [15]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Lambertmouth, user_id=fd33b38d-c31a-42b9-88ec-09dbcdf804f9>
Query user_metris: UserMetrics <city=North Daniel, user_id=63065224-f174-4cdc-be5b-a64f685684f2>
Query user_metris: UserMetrics <city=Port Rebecca, user_id=417930eb-ae2c-4a54-8e36-f6ee8e08a69f>
Query user_metris: UserMetrics <city=Lukefort, user_id=50a34ebb-b32e-45e1-90eb-3dbc1822879c>
Query user_metris: UserMetrics <city=East Anna, user_id=895d6404-8993-4a7e-a755-25882ff88059>
Query user_metris: UserMetrics <city=Fischermouth, user_id=de5b4289-0dd5-4dff-ac1a-44500b22e485>
Query user_metris: UserMetrics <city=North Elizabeth, user_id=a59f09c1-b855-4495-9941-4e75f846be58>
Query user_metris: UserMetrics <city=North Xaviermouth, user_id=4c8b80b5-22e6-45c2-b8c9-7abf6fbfd9d9>
Query user_metris: UserMetrics <city=Rebeccabury, user_id=3da742fe-6bc2-4cc9-ac07-05a7fc7e83a6>
Query user_metris: UserMetrics <city=Serranoport, user_id=e6f7c1e6-202b-48a2-b660-02a180819744>
Query user_metris: UserMetrics 

### Insert/Create

In [16]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Barrerachester', user_id=UUID('9e90e209-cb4c-4abe-98a3-cea53a816484'), username='scottjoshua', session_duration=655, last_access=datetime.datetime(1980, 10, 20, 16, 40, 14))

In [17]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [18]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering()
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=North Daniel, user_id=63065224-f174-4cdc-be5b-a64f685684f2>
Delete user_metris: UserMetrics <city=Port Rebecca, user_id=417930eb-ae2c-4a54-8e36-f6ee8e08a69f>
Delete user_metris: UserMetrics <city=Valentineshire, user_id=1f8da13c-f2ad-4601-801c-e3efa2e2bdc8>
Delete user_metris: UserMetrics <city=Lukefort, user_id=50a34ebb-b32e-45e1-90eb-3dbc1822879c>
Delete user_metris: UserMetrics <city=East Anna, user_id=895d6404-8993-4a7e-a755-25882ff88059>
Delete user_metris: UserMetrics <city=Brownport, user_id=4cbc1f19-0925-4277-ace0-71c25df77161>
Delete user_metris: UserMetrics <city=Fischermouth, user_id=de5b4289-0dd5-4dff-ac1a-44500b22e485>
Delete user_metris: UserMetrics <city=North Elizabeth, user_id=a59f09c1-b855-4495-9941-4e75f846be58>
Delete user_metris: UserMetrics <city=North Xaviermouth, user_id=4c8b80b5-22e6-45c2-b8c9-7abf6fbfd9d9>
Delete user_metris: UserMetrics <city=Rebeccabury, user_id=3da742fe-6bc2-4cc9-ac07-05a7fc7e83a6>
Delete user_metris: U

In [19]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")